# Wow, such empty

## Imports yeah

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Generator lol

In [ ]:
# Constants
volume_HCl = 50  # Volume of HCl in mL
NaOH_concentration = 0.1  # Molar concentration of NaOH
volumes_added = np.linspace(0, 100, 1000)  # Volume of NaOH added in mL (0 to 100 mL)

# Directory to store generated files
output_dir = "titration_datasets"
os.makedirs(output_dir, exist_ok=True)

# Function to calculate pH
def calculate_pH(HCl_conc, NaOH_conc, V_HCl, V_NaOH_added):
    # Moles of HCl initially
    moles_HCl = HCl_conc * V_HCl / 1000  # in moles
    
    # Moles of NaOH added
    moles_NaOH = NaOH_conc * V_NaOH_added / 1000  # in moles

    # Before equivalence point: excess H+
    if moles_NaOH < moles_HCl:
        moles_H_remaining = moles_HCl - moles_NaOH
        concentration_H = moles_H_remaining / (V_HCl + V_NaOH_added) * 1000
        pH = -np.log10(concentration_H)
    
    # At equivalence point: pH = 7 (neutral solution)
    elif np.isclose(moles_NaOH, moles_HCl, atol=1e-5):
        pH = 7
    
    # After equivalence point: excess OH-
    else:
        moles_OH_excess = moles_NaOH - moles_HCl
        concentration_OH = moles_OH_excess / (V_HCl + V_NaOH_added) * 1000
        pOH = -np.log10(concentration_OH)
        pH = 14 - pOH

    return pH

# Generate titration curves for 100 datasets
for i in range(1, 101):
    # Randomize concentration of HCl for each titration between 0.05 M and 1.0 M
    HCl_concentration = np.random.uniform(0.05, 1.0)
    
    # Store pH values
    pH_values = []

    for V_NaOH in volumes_added:
        pH = calculate_pH(HCl_concentration, NaOH_concentration, volume_HCl, V_NaOH)
        pH_values.append(pH)
    
    # Create DataFrame
    df = pd.DataFrame({
        "Volume NaOH (mL)": volumes_added,
        "pH": pH_values
    })
    
    # Add some random noise to simulate real data (small fluctuations in pH)
    df["pH"] = df["pH"] + np.random.normal(0, 0.05, df["pH"].shape)
    
    # Save to CSV file
    df.to_csv(os.path.join(output_dir, f"titration_{i}.csv"), index=False)

print("Titration data generated successfully!")

## Sexy data is sexy

In [ ]:
# Directory containing the titration datasets
data_dir = "titration_datasets"

# Initialize plot
plt.figure(figsize=(10, 6))

# Loop through the CSV files and plot each titration curve
for filename in os.listdir(data_dir):
    if filename.endswith(".csv"):
        filepath = os.path.join(data_dir, filename)
        df = pd.read_csv(filepath)
        
        # Plot titration curve
        plt.plot(df["Volume NaOH (mL)"], df["pH"], label=filename.split('_')[1], alpha=0.5)

# Add labels and title
plt.xlabel("Volume of NaOH added (mL)")
plt.ylabel("pH")
plt.title("Simulated Titration Curves")
plt.grid(True)

# Show plot
plt.tight_layout()
plt.show()